In [122]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [123]:
def lazy_scroll(driver):
    last_height = driver.execute_script('return document.body.scrollHeight')
    move_to = 3000

    while True:
        driver.execute_script(f'scrollTo(0,{move_to})')
        time.sleep(1)

        new_height = driver.execute_script('return document.body.scrollHeight')

        if new_height == last_height:
            break

        last_height = new_height
        move_to += 3000

In [124]:
titles = []
prices = []
links  = []

In [125]:
def aliexpress_page_scraper(html):

    soup = BeautifulSoup(html , 'html.parser')
    products = soup.find_all('div' , class_ = 'kr_k3')

    for product in products:

        product_title = product.find('div', class_ = 'kr_av').get('title')
        product_price = product.find('div', class_ = 'kr_kj').text.replace(',' , '')
        product_link  = product.find('a', class_ = 'kr_b in_is search-card-item').get('href')

        titles.append(product_title)
        prices.append(float(product_price[3:]))
        links.append(product_link)

In [126]:
def close_add(driver):
    try:
        close_button = driver.find_element(By.CLASS_NAME, 'esm--upload-close--1x0SREz')
        close_button.click()
    except:
        print('No ad.')

In [127]:
url = 'https://www.aliexpress.com/w/wholesale-airpods.html'
driver = webdriver.Chrome()

driver.get(url)

print(driver.title)

page = 1

while page <= 5:

    time.sleep(2)
    lazy_scroll(driver)
    aliexpress_page_scraper(driver.page_source)
    next_btn = driver.find_element(By.CLASS_NAME, 'comet-pagination-next')
    page += 1
    
    if next_btn.get_attribute('aria-disabled') == 'false':
        driver.execute_script("arguments[0].click();", next_btn)
        print("Button Clicked")
    else:
        break

driver.quit()

Captcha Interception


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".comet-pagination-next"}
  (Session info: chrome=137.0.7151.56); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff604c1fea5+79173]
	GetHandleVerifier [0x0x7ff604c1ff00+79264]
	(No symbol) [0x0x7ff6049d9e5a]
	(No symbol) [0x0x7ff604a30586]
	(No symbol) [0x0x7ff604a3083c]
	(No symbol) [0x0x7ff604a84247]
	(No symbol) [0x0x7ff604a589af]
	(No symbol) [0x0x7ff604a8100d]
	(No symbol) [0x0x7ff604a58743]
	(No symbol) [0x0x7ff604a214c1]
	(No symbol) [0x0x7ff604a22253]
	GetHandleVerifier [0x0x7ff604eea2dd+3004797]
	GetHandleVerifier [0x0x7ff604ee472d+2981325]
	GetHandleVerifier [0x0x7ff604f03380+3107360]
	GetHandleVerifier [0x0x7ff604c3aa2e+188622]
	GetHandleVerifier [0x0x7ff604c422bf+219487]
	GetHandleVerifier [0x0x7ff604c28df4+115860]
	GetHandleVerifi

In [ ]:
df = pd.DataFrame({
    'Titles' : titles,
    'Prices' : prices,
    'Links'  : links
})

df.shape

(60, 3)

In [ ]:
df = df[df['Links'].str.contains('item')]
df.reset_index( drop=True , inplace=True)
df.index = range(1,len(df)+1)
df.shape

(50, 3)

In [ ]:
df.to_csv('aliexpress_products.csv')
df.head()

,Titles,Prices,Links
1,Silicone Protective Case Retro Pager Beeper 3D...,760.17,//www.aliexpress.com/item/1005008799323267.htm...
2,Korean Style Flower Stereoscopic Ribbon Bowkno...,1643.66,//www.aliexpress.com/item/1005009147058881.htm...
3,Cover for AirPods Pro 2 Wireless Bluetooth Ear...,657.05,//www.aliexpress.com/item/1005006630268718.htm...
4,for airpod 3 2021 case Silicone Soft Skin Prot...,641.64,//www.aliexpress.com/item/1005006438445350.htm...
5,3-piece Set For Apple AirPods 4 Case With Clea...,740.81,//www.aliexpress.com/item/1005008932731131.htm...
